In [21]:
#imports
import pandas as pd
import numpy as np
import re
import collections
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package wordnet to /Users/kevinha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
#The 900-element dataset was split into a training set used to train the network and a testing set that further EDA was performed upon
df2=pd.read_csv('balanced_dataset_training.csv', encoding='latin')
df_test=pd.read_csv('balanced_dataset_testing.csv', encoding='latin')

In [52]:
#The four columns- "company_profile, description, requirements, benefits" were combined(because many contain null values)
df2.head()

,Unnamed: 0,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,421,Mobile Engineer,"US, ND, Bismarck",,,<p>We're changing how the world creates and pu...,<p>We're changing how the world creates and pu...,<h3>The Musts:</h3>\r\n<ul>\r\n<li>You think b...,<h3>The Perks (full-time only):</h3>\r\n<ul>\r...,f,t,t,Full-time,Entry level,Unspecified,Program Development,Other,f,t
1,545,BI Consultant & Database Expert,"BE, ,",tech,,<p>hello world</p>\r\n<p>talents23_ drives the...,<p>hello world</p>\r\n<p>talents23_ drives the...,<ul>\r\n<li>You know SQL and have professional...,<ul>\r\n<li>WeÃÂ¢ÃÂÃÂre theÃÂÃÂ key An...,f,t,t,Full-time,Associate,,,,f,t
2,806,Customer Service Representative,"US, TX, SAN ANTONIO",,,,<p>SUMMARYResponsible for acting as a liaison...,<p>Experience: Proficiency with Microsoft Word...,<p>You would be eligible to Benefits after 2mo...,f,f,f,Full-time,Not Applicable,High School or equivalent,Consumer Services,Customer Service,t,t
3,803,Administrative Assistant,"US, CA, Santa Ana",,,,<p>ÃÂÃÂ In addition to clerical and admini...,,<p>No requirements</p>,f,f,f,Full-time,,,Accounting,Administrative,t,t
4,147,Key Account Manager Switzerland,"CH, BS, Basel",Sales,,,<p>Are you looking for a career that will cha...,,,f,t,t,Full-time,Associate,Bachelor's Degree,Education Management,Sales,f,t


In [53]:
df2.isna().sum()

Unnamed: 0             0
title                  0
location               0
department             0
salary_range           0
company_profile        0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
in_balanced_dataset    0
dtype: int64

In [54]:
df2 = df2.fillna("")
df2['description']=df2['company_profile']+" "+df2['description']+" "+ df2['requirements']+" "+df2['benefits']
df2

,Unnamed: 0,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,in_balanced_dataset
0,421,Mobile Engineer,"US, ND, Bismarck",,,<p>We're changing how the world creates and pu...,<p>We're changing how the world creates and pu...,<h3>The Musts:</h3>\r\n<ul>\r\n<li>You think b...,<h3>The Perks (full-time only):</h3>\r\n<ul>\r...,f,t,t,Full-time,Entry level,Unspecified,Program Development,Other,f,t
1,545,BI Consultant & Database Expert,"BE, ,",tech,,<p>hello world</p>\r\n<p>talents23_ drives the...,<p>hello world</p>\r\n<p>talents23_ drives the...,<ul>\r\n<li>You know SQL and have professional...,<ul>\r\n<li>WeÃÂ¢ÃÂÃÂre theÃÂÃÂ key An...,f,t,t,Full-time,Associate,,,,f,t
2,806,Customer Service Representative,"US, TX, SAN ANTONIO",,,,<p>SUMMARYResponsible for acting as a liaiso...,<p>Experience: Proficiency with Microsoft Word...,<p>You would be eligible to Benefits after 2mo...,f,f,f,Full-time,Not Applicable,High School or equivalent,Consumer Services,Customer Service,t,t
3,803,Administrative Assistant,"US, CA, Santa Ana",,,,<p>ÃÂÃÂ In addition to clerical and admin...,,<p>No requirements</p>,f,f,f,Full-time,,,Accounting,Administrative,t,t
4,147,Key Account Manager Switzerland,"CH, BS, Basel",Sales,,,<p>Are you looking for a career that will ch...,,,f,t,t,Full-time,Associate,Bachelor's Degree,Education Management,Sales,f,t
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,718,Data Entry,"US, FL, HILLIARD FL",,25-30,,<p>Prepares source data for computer entry b...,<p>We are seeking extremely motivated and expe...,"<p>ÃÂÃÂ Health, Dental, Life and AD&amp;D I...",f,f,f,Contract,Internship,,Consumer Services,Accounting/Auditing,t,t
446,826,Personal Assistant,"US, NY, New York",Admin - Clerical,,,<p>Job Description</p>\r\n<p><br>Vertex Glob...,,,f,f,f,Temporary,Entry level,Unspecified,Telecommunications,Administrative,t,t
447,675,OR Specialty Coordinator,"US, ,",,,,<p><b>Apply using below link</b></p>\r\n<p><...,,,f,f,f,Full-time,,,Hospital & Health Care,,t,t
448,555,Sales Consultant NYC,"US, NY, New York",,,"<p>Sarbari is a high-growth, investment-backed...","<p>Sarbari is a high-growth, investment-backed...",,<p>Sarbari offers a base salary and high earni...,f,t,f,Full-time,Associate,,Food & Beverages,Sales,f,t


In [56]:
#extracting the description and fraudulent columns
def fraudulent(x):
    if(x=='f'):
        return 0
    return 1
df_OD = df2[['fraudulent', 'description']].copy()
df_OD.head()
df_OD['scam'] = df2['fraudulent'].map(lambda x:fraudulent(x))
df_OD['text']= df2['description'].copy()
df_OD.head()

,fraudulent,description,scam,text
0,f,<p>We're changing how the world creates and pu...,0,<p>We're changing how the world creates and pu...
1,f,<p>hello world</p>\r\n<p>talents23_ drives the...,0,<p>hello world</p>\r\n<p>talents23_ drives the...
2,t,<p>SUMMARYResponsible for acting as a liaiso...,1,<p>SUMMARYResponsible for acting as a liaiso...
3,t,<p>ÃÂÃÂ In addition to clerical and admin...,1,<p>ÃÂÃÂ In addition to clerical and admin...
4,f,<p>Are you looking for a career that will ch...,0,<p>Are you looking for a career that will ch...


In [27]:
#dropping columns, leaving "scam" and "text"
df_OD=df_OD.drop(['fraudulent', 'description'], axis=1)

df_OD.head()
    


,scam,text
0,0,<p>We're changing how the world creates and pu...
1,0,<p>hello world</p>\r\n<p>talents23_ drives the...
2,1,<p>SUMMARYResponsible for acting as a liaison...
3,1,<p>ÃÂÃÂ In addition to clerical and admini...
4,0,<p>Are you looking for a career that will cha...


In [57]:
#lemmatization of data using library, removal of HTML tags
lem = WordNetLemmatizer()
def preprocessing(data):
      sms = contractions.fix(data) # converting shortened words to original (Eg:"I'm" to "I am")
      sms = sms.lower() # lower casing the sms
      sms = re.sub(r'https?://S+|www.S+', "", sms).strip() #removing url
      sms = re.sub("[^a-z ]", "", sms) # removing symbols and numbes
      sms = sms.split() #splitting
      # lemmatization and stopword removal
      sms = [lem.lemmatize(word) for word in sms if not word in set(stopwords.words("english"))]
      sms = " ".join(sms)
      return sms

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result


df_OD['text']=df_OD['text'].apply(lambda cw : remove_tags(cw))

X = df_OD["text"].apply(preprocessing)


In [58]:
#encode "scam" column into array
from sklearn.preprocessing import LabelEncoder
lb_enc = LabelEncoder()
y = lb_enc.fit_transform(df_OD["scam"])

In [59]:
#tokenize data, convert to integers
tokenizer = Tokenizer() #initializing the tokenizer
tokenizer.fit_on_texts(X)# fitting on the sms data
text_to_sequence = tokenizer.texts_to_sequences(X)
print(X)

0      changing world creates publishes content even ...
1      hello worldtalents drive change digital recrui...
2      summaryresponsible acting liaison customer com...
3      addition clerical administrative duty individu...
4      looking career challenge allow make impact eve...
                             ...                        
445    prepares source data computer entry compiling ...
446    job descriptionvertex global solution seek exe...
447    apply using linkurlabecbbbaadffccccaaebor spec...
448    sarbari highgrowth investmentbacked software c...
449    healthlink provides computer communication sys...
Name: text, Length: 450, dtype: object


In [60]:
#add padding, the length of every sequence is the max length
max_length_sequence = max([len(i) for i in text_to_sequence])
 # finding the length of largest sequence
padded_sms_sequence = pad_sequences(text_to_sequence, maxlen=max_length_sequence, 
                                    padding = "pre") 
len(padded_sms_sequence)

450

In [65]:
#LSTM MODEL, utilizing dropouts to prevent overfitting
TOT_SIZE = len(tokenizer.word_index)+1
def create_model():
      lstm_model = Sequential()
      lstm_model.add(Embedding(TOT_SIZE, 32, input_length=max_length_sequence))
      lstm_model.add(LSTM(50))
      lstm_model.add(Dropout(0.4))
      lstm_model.add(Dense(20, activation="relu"))
      lstm_model.add(Dropout(0.3))
      lstm_model.add(Dense(1, activation = "sigmoid"))
      return lstm_model
lstm_model = create_model()
lstm_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["binary_crossentropy"])
lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 4024, 32)          407232    
                                                                 
 lstm_3 (LSTM)               (None, 50)                16600     
                                                                 
 dropout_6 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 20)                1020      
                                                                 
 dropout_7 (Dropout)         (None, 20)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 424873 (1.62 MB)
Trainable params: 42487

In [66]:
#batch-size was determined to be 64, around 10 epochs produced most balanced results
history=lstm_model.fit(padded_sms_sequence, y, epochs =9, validation_split=0.2, batch_size=64)

Epoch 1/9
6/6 [==============================] - 12s 2s/step - loss: 0.6923 - binary_crossentropy: 0.6923 - val_loss: 0.6906 - val_binary_crossentropy: 0.6906
Epoch 2/9
6/6 [==============================] - 10s 2s/step - loss: 0.6887 - binary_crossentropy: 0.6887 - val_loss: 0.6879 - val_binary_crossentropy: 0.6879
Epoch 3/9
6/6 [==============================] - 10s 2s/step - loss: 0.6793 - binary_crossentropy: 0.6793 - val_loss: 0.6807 - val_binary_crossentropy: 0.6807
Epoch 4/9
6/6 [==============================] - 10s 2s/step - loss: 0.6544 - binary_crossentropy: 0.6544 - val_loss: 0.6530 - val_binary_crossentropy: 0.6530
Epoch 5/9
6/6 [==============================] - 10s 2s/step - loss: 0.5463 - binary_crossentropy: 0.5463 - val_loss: 0.5723 - val_binary_crossentropy: 0.5723
Epoch 6/9
6/6 [==============================] - 10s 2s/step - loss: 0.4428 - binary_crossentropy: 0.4428 - val_loss: 0.5038 - val_binary_crossentropy: 0.5038
Epoch 7/9
6/6 [==============================]

In [68]:
#convert testing data in the same manner as the training data

df_test = df_test.fillna("")
df_test['description']=df_test['company_profile']+df_test['description']
df_OD_test = df_test[['fraudulent', 'description']].copy()
df_OD_test.head()
df_OD_test['scam'] = df_test['fraudulent'].map(lambda x:fraudulent(x))
df_OD_test['text']= df_test['description'].copy()
df_OD_test=df_OD_test.drop(['fraudulent', 'description'], axis=1)
df_OD_test['text']=df_OD_test['text'].apply(lambda cw : remove_tags(cw))
X = df_OD_test["text"].apply(preprocessing)
for i in range(5):
           print("Text               : ",X[i] )
           print("Numerical Sequence : ", text_to_sequence[i])
tokenizer = Tokenizer() #initializing the tokenizer
tokenizer.fit_on_texts(X)# fitting on the sms data
text_to_sequence = tokenizer.texts_to_sequences(X)

 # finding the length of largest sequence
padded_sms_sequence_test = pad_sequences(text_to_sequence, maxlen=max_length_sequence, 
                                    padding = "pre") 
padded_sms_sequence_test


Text               :  accepting online application onlyclick applythis full time temporary position lasting yearsdescriptionvalidate review legal contractual agreement customer input contract contract database contract completed amp reviewed within perdetermined service level agreement professional email interaction customer scanning uploading documentsqualificationshigh school diploma equivalent professional communication skill via email interactiondedicated need business project management skill assist facilitating multiple contract reject detail oriented able multitaskability work time sensitive document must able work independently able perform team environment needed fast accurate typistaccepting online application onlyclick apply
Numerical Sequence :  [697, 118, 1406, 4886, 261, 698, 4887, 932, 4888, 898, 228, 4074, 110, 58, 697, 118, 1406, 4886, 261, 698, 4887, 932, 4888, 898, 228, 4074, 110, 58, 37, 130, 43, 361, 933, 187, 9090, 110, 997, 248, 161, 9091, 202, 83, 69, 2024, 1487

array([[   0,    0,    0, ...,   52,  998,  162],
       [   0,    0,    0, ...,   21,  282, 1526],
       [   0,    0,    0, ..., 1332, 1333, 1334],
       ...,
       [   0,    0,    0, ...,  849, 1010, 1780],
       [   0,    0,    0, ...,  824,   14,   38],
       [   0,    0,    0, ...,    3,  415,  980]], dtype=int32)

In [69]:
#encode test data
lb_enc = LabelEncoder()
y_test = lb_enc.fit_transform(df_OD_test["scam"])

In [70]:
len(padded_sms_sequence_test)

450

In [86]:
#generate predictions for testing data, take values with probabliltiy above median as spam
pred = lstm_model.predict(padded_sms_sequence_test)
y_pred=[]
med= np.median(pred)
print(med)
for i in range(0,pred.size):
    if pred[i][0] >med:
        y_pred.append(1)
    else:
        y_pred.append(0)
from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, y_pred)

15/15 [==============================] - 3s 226ms/step
0.93326724


array([[179,  52],
       [ 46, 173]])

In [72]:
#generate histogram for probability distribution
hist,bins = np.histogram(pred,bins = [0,0.2,0.4,0.6,0.8,1]) 

In [73]:
hist

array([  0,  25, 131,  35, 259])

In [74]:
#export model to pickle
import pickle
pickle.dump(lstm_model, open('model3.pkl', 'wb'))

In [77]:
#load model from pickle
model = pickle.load(open('model3.pkl','rb'))
pred = model.predict(padded_sms_sequence_test)

y_pred=[]
y_column=[]
med= np.median(pred)
print(med)
for i in range(0,pred.size):
    if pred[i][0] >med:
        y_pred.append(1)
    else:
        y_pred.append(0)
    y_column.append(pred[i][0])
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

15/15 [==============================] - 3s 203ms/step
0.93326724


array([[179,  52],
       [ 46, 173]])

In [78]:
#create new column in dataset with predictions
df_test['LSTM_PREDICT'] = y_pred

In [79]:
df_test['LSTM_PROB']=y_column

In [80]:
#send to csv
df_test.to_csv('EDA_Dataset.csv',index=False)

In [81]:
df_works=pd.read_csv('EDA_Dataset.csv')

In [82]:
df_true=df_works[df_works['fraudulent']=='t']

In [83]:
#get average probability of spam of actual spam
sum(df_true['LSTM_PROB'])/len(df_true['LSTM_PROB'])

0.8926078148401827

In [84]:
df_false=df_works[df_works['fraudulent']=='f']

In [85]:
#get average probability of spam of not spam
sum(df_false['LSTM_PROB'])/len(df_false['LSTM_PROB'])

0.6212031174025973

In [ ]:
#Note: An LSTM is rather chaotic for the predicitons it generates and is very senseitive to initial conditions. The dataset generated by the LSTM in this code is different frmo the one used in the final product.
#The results that were selected had a median of 45.
#However, the difference between the predicted spam and not spam averages is typically around 25%.